In [1]:
!pip install scikit-learn
!pip install --upgrade pixiedust

Requirement already up-to-date: pixiedust in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.1.18)


In [2]:
import pixiedust
import sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import arff
import brunel
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Pixiedust database opened successfully


In [3]:
raw_df=pixiedust.sampleData('https://raw.githubusercontent.com/apischdo/skillsacademy/master/Denormalized%20claims%20data.csv')

Downloaded 463947 bytes
Creating pandas DataFrame for 'https://raw.githubusercontent.com/apischdo/skillsacademy/master/Denormalized%20claims%20data.csv'. Please wait...
Loading file using 'pandas'
Successfully created pandas DataFrame for 'https://raw.githubusercontent.com/apischdo/skillsacademy/master/Denormalized%20claims%20data.csv'


In [4]:
display(raw_df)

HOUSEHOLD_ID,DRIVER_ID,POLICY_ID,CLAIM_ID,INCIDENT_CAUSE,DESCRIPTION,CLAIM_STATUS,ODOMETER_AT_LOSS,LOSS_EVENT_TIME,CLAIM_INIT_TIME,POLICE_REPORT,CLAIMS_AT_LOSS_DATE,LOSS_LOCATION_LAT,LOSS_LOCATION_LONG,CLAIM_AMOUNT,FLAG_FOR_FRAUD_INV,PRIMARY_DRIVER_ID,START_DATE,EXPIRY_DATE,MODEL_YEAR,MAKE,MODEL,PLATE,COLOR,INITIAL_ODOMETER,LOW_MILEAGE_USE,FIRST_NAME,LAST_NAME,GENDER,BIRTHDATE,SSN,DRIVERS_LICENSE_ID,DRIVERS_LICENSE_EXPIRY,DRIVERS_LICENSE_STATE,DATE_AT_CURRENT_ADDRESS,CONTACT_NUMBER,EMAIL,COMMUTE_DISCOUNT,WORK_ADDR_NUMB,WORK_ADDR_STREET_NAME,WORK_ADDR_UNIT_DESIGNATION,WORK_ADDR_UNIT_NUMBER,WORK_ADDR_CITY,WORK_ADDR_STATE,WORK_ADDR_ZIPCODE,WORK_ADDR_LATITUDE,WORK_ADDR_LONGITUDE,WORK_ADDR_PBKEY,ADDR_STREET_NUMBER,ADDR_STREET_NAME,ADDR_UNIT_DESIGNATION,ADDR_UNIT_NUMBER,CITY,STATE,ZIPCODE,LATITUDE,LONGITUDE,PBKEY
AI11900,CCS4638,YR5487331,A-2018-KD410,2,nan,3,148503.3,2018-01-10 00:00:00.0,2018-01-17 00:00:00.0,0,1,41.85242413,-87.63599569,2090.0,0,CCS4638,2017-07-09,2018-07-09,2011,Hyundai,Sonata,UXPY795,Med Red,139848.0,0,Carlene,Lepp,F,1974-06-25,720-76-8886,N597-4580-2360,2019-05-08,IL,2008-05-23,749-213-0908,clepp@inboux.com,0,15306,PAGE AVE,nan,nan,HARVEY,IL,60426,41.611456,-87.659924,P00007UWCVT5,260,E CHESTNUT ST,APT,2209,CHICAGO,IL,60611,41.898826,-87.619601,-87.619601
LM58734,KPQ4157,YX8982998,A-2018-TT969,3,nan,3,245549.0,2018-03-20 00:00:00.0,2018-03-24 00:00:00.0,0,4,41.92834542,-87.6684029,1180.0,0,KPQ4157,2017-06-02,2018-06-02,2008,Buick,Enclave,RRHN760,White,232920.0,0,Horace,Mathesius,M,1961-10-11,869-70-2920,G840-4521-6716,2019-11-13,IL,2007-01-25,348-729-0078,hmathesius@yahoot.com,0,417,THAMES DR,nan,nan,SHOREWOOD,IL,60404,41.527642,-88.22166,P00007UTPQES,7021,HILLCREST DR,nan,nan,DOWNERS GROVE,IL,60516,41.760323,-88.019281,-88.019281
DG34465,CPU1132,VR8204534,A-2016-WM469,5,nan,3,217958.2,2016-04-19 00:00:00.0,2016-04-30 00:00:00.0,1,3,41.83708346,-87.66649329,2323.0,0,HSW2671,2015-05-25,2016-05-24,2007,Acura,MDX,EUHZ076,Silver,208008.0,0,Maria,Buttz,F,1967-11-15,451-99-0985,Q111-1930-9883,2018-04-12,IL,2013-06-11,237-947-0767,mbuttz@outloop.com,0,630,VERNON AVE,APT,4,GLENCOE,IL,60022,42.132261,-87.758721,P00007UT0G0F,40557,N TRINITY LN,nan,nan,ANTIOCH,IL,60002,42.45203,-88.108412,-88.108412
FH89848,LKU4949,TO6968240,A-2017-JY910,5,nan,3,178289.7,2017-06-06 00:00:00.0,2017-06-17 00:00:00.0,0,1,41.83804345,-87.72574496,2967.0,0,LKU4949,2016-05-13,2017-05-13,2009,Toyota,Highlander,ZZNM057,Med Red,165540.0,0,Lona,Inthisone,F,1968-10-08,615-02-6017,D760-7141-5773,2020-03-11,IL,2013-08-19,241-856-0642,linthisone@ggmail.com,1,1459,BERKLEY CT,nan,nan,DEERFIELD,IL,60015,42.178422,-87.861797,P00007USZ9L7,3916,SUNSET AVE,nan,nan,MARKHAM,IL,60428,41.597581,-87.712766,-87.712766
KL55170,EQR6564,YQ8086097,A-2018-CB281,3,nan,1,264172.3,2018-05-24 00:00:00.0,2018-05-30 00:00:00.0,0,1,41.75592679,-87.65402522,44033.5,1,EQR6564,2017-11-03,2018-11-03,2007,Kia,Sorento,JBRA754,Silver,257328.0,0,Bob,Rozenboom,M,1942-10-12,670-97-0031,A906-4838-9382,2021-09-11,IL,2006-05-21,745-959-0748,brozenboom@yahoot.com,0,2001,LAKEVIEW DR,nan,nan,WHEATON,IL,60189,41.832091,-88.14451700000001,P00007UTMJOE,1656,W FARWELL AVE,APT,3F,CHICAGO,IL,60626,42.006924,-87.672253,-87.672253
MK16710,KBA5650,NP8088051,A-2016-ME516,5,nan,3,80957.8,2016-03-03 00:00:00.0,2016-03-08 00:00:00.0,1,1,41.74933329,-87.5827846,1700.0,0,KBA5650,2015-09-09,2016-09-08,2012,Kia,Sportage,YXYY904,Silver,79632.0,0,Guillermo,Tsung,M,1995-06-22,622-98-8994,C668-9766-5688,2019-06-07,IL,2014-02-13,741-670-0732,gtsung@gmxx.com,0,1430,S KEELER AVE,nan,nan,CHICAGO,IL,60623,41.861491,-87.730266,P00007UWEQQG,5118,W MELROSE ST,nan,nan,CHICAGO,IL,60641,41.939901,-87.755131,-87.755131
LH46463,JUN3571,ZS4755633,A-2017-ZK342,3,nan,3,260612.0,2017-10-26 00:00:00.0,2017-10-29 00:00:00.0,1,3,41.79591839,-87.76178079,3139.5,0,JUN3571,2016-01-27,2017-01-26,2005,Toyota,Corolla,IEQJ110,Med. Dark Gray,249636.0,0,Dolores,Rohne,F,1942-08-01,511-37-4339,G308-3537-6868,2018-07-09,IL,2000-09-25,736-259-0

In [5]:
# Claim within 15 days of policy expiry (date of loss - insurance_policy.expiry)
raw_df["EXPIRY_DATE"] = pd.to_datetime(raw_df["EXPIRY_DATE"])
raw_df["LOSS_EVENT_TIME"] = pd.to_datetime(raw_df["LOSS_EVENT_TIME"])

raw_df["DAYS_FROM_LOSS"] = raw_df["LOSS_EVENT_TIME"] - raw_df["EXPIRY_DATE"]
raw_df["DAYS_FROM_LOSS"] = abs(raw_df.DAYS_FROM_LOSS.dt.days)

raw_df.loc[raw_df['DAYS_FROM_LOSS'] >= 15, 'SUSPICIOUS_CLAIM_TIME'] = 1
raw_df.loc[raw_df['DAYS_FROM_LOSS'] < 15, 'SUSPICIOUS_CLAIM_TIME'] = 0

In [6]:
raw_df["SUSPICIOUS_CLAIM_TIME"].value_counts()

1.0    915
0.0     60
Name: SUSPICIOUS_CLAIM_TIME, dtype: int64

In [7]:
# Expired drivers license (if date of loss > insurance_driver.drivers_license_expiry)
raw_df["DRIVERS_LICENSE_EXPIRY"] = pd.to_datetime(raw_df["DRIVERS_LICENSE_EXPIRY"])

raw_df["DAYS_FROM_L_EXPIRY"] = raw_df["DRIVERS_LICENSE_EXPIRY"] - raw_df["LOSS_EVENT_TIME"]
raw_df["DAYS_FROM_L_EXPIRY"] = raw_df.DAYS_FROM_L_EXPIRY.dt.days

raw_df.loc[raw_df['DAYS_FROM_L_EXPIRY'] >= 0, 'EXPIRED_LICENSE'] = 0
raw_df.loc[raw_df['DAYS_FROM_L_EXPIRY'] < 0, 'EXPIRED_LICENSE'] = 1

In [8]:
# Days living at current address (date of loss - insurance_driver.date_at_current_address)

raw_df["DATE_AT_CURRENT_ADDRESS"] = pd.to_datetime(raw_df["DATE_AT_CURRENT_ADDRESS"])
raw_df["DAYS_AT_ADDRESS"] = raw_df["LOSS_EVENT_TIME"] - raw_df["DATE_AT_CURRENT_ADDRESS"]
raw_df["DAYS_AT_ADDRESS"] = abs(raw_df.DAYS_AT_ADDRESS.dt.days)

raw_df.loc[raw_df['DAYS_AT_ADDRESS'] >= 15, 'SUSPICIOUS_LIVING'] = 1
raw_df.loc[raw_df['DAYS_AT_ADDRESS'] < 15, 'SUSPICIOUS_LIVING'] = 0

In [9]:
raw_df["SUSPICIOUS_LIVING"].value_counts()

1.0    973
0.0      2
Name: SUSPICIOUS_LIVING, dtype: int64

In [10]:
#7500/year
raw_df["START_DATE"] = pd.to_datetime(raw_df["START_DATE"])
#find number of days between policy creation and accident
raw_df["LENGTH_OF_POLICY"]=(raw_df["LOSS_EVENT_TIME"] - raw_df["START_DATE"]).dt.days

#convert to years
raw_df["LENGTH_OF_POLICY"]=raw_df["LENGTH_OF_POLICY"]/365

#divide Odometer at loss by years
raw_df["MILES/YEAR"] = raw_df["ODOMETER_AT_LOSS"]/raw_df["LENGTH_OF_POLICY"]
raw_df["MILES/YEAR"].value_counts()

145432.118234    1
234953.253233    1
55515.574648     1
123525.401515    1
198472.566993    1
41161.635017     1
316044.542945    1
343652.573991    1
357034.135714    1
444934.671171    1
47761.055147     1
106608.700207    1
280168.774737    1
97407.709325     1
83715.343421     1
836054.660714    1
225418.465704    1
119793.802198    1
49174.210227     1
154269.516340    1
87012.862810     1
55960.766821     1
340284.423841    1
55199.213855     1
158134.051821    1
148190.243875    1
128341.984375    1
388218.025735    1
38327.581707     1
311332.643229    1
                ..
138485.866667    1
355749.475610    1
206160.525547    1
172133.553061    1
39974.457997     1
129429.385563    1
227492.647193    1
581033.280000    1
213403.812808    1
129338.850177    1
433650.444700    1
428016.279255    1
102579.963602    1
924887.694444    1
258935.458015    1
219977.172330    1
167467.854637    1
110428.874687    1
140508.777778    1
189399.690217    1
54898.636569     1
42573.963602

In [11]:
# Conflict on whether a policyholder with a low mileage discount experienced a loss with high mileage at the point of loss
raw_df.loc[raw_df["MILES/YEAR"] <7500, 'LOW_MILEAGE_AT_LOSS'] = 1
raw_df.loc[raw_df["MILES/YEAR"] >=7500, 'LOW_MILEAGE_AT_LOSS'] = 0

In [12]:
raw_df.loc[raw_df["LOW_MILEAGE_USE"]==raw_df["LOW_MILEAGE_AT_LOSS"], 'SUSPICIOUS_MILEAGE'] = 0
raw_df.loc[raw_df["LOW_MILEAGE_USE"]!=raw_df["LOW_MILEAGE_AT_LOSS"], 'SUSPICIOUS_MILEAGE'] = 1

In [13]:
raw_df.loc[raw_df["CLAIM_AMOUNT"] <3000, 'EXCESSIVE_CLAIM_AMOUNT'] = 0
raw_df.loc[raw_df["CLAIM_AMOUNT"] >=3000, 'EXCESSIVE_CLAIM_AMOUNT'] = 1

In [17]:
# dataframes for certain features
features = ['FLAG_FOR_FRAUD_INV', 
 'SUSPICIOUS_MILEAGE',
 'EXPIRED_LICENSE',
 'SUSPICIOUS_CLAIM_TIME',
 'SUSPICIOUS_LIVING',
 'EXCESSIVE_CLAIM_AMOUNT']

In [18]:
df_model = raw_df[features]

In [19]:
#ensure all relevant features are integers
df_model["SUSPICIOUS_LIVING"] = df_model["SUSPICIOUS_LIVING"].astype(int)
df_model["EXPIRED_LICENSE"] = df_model["EXPIRED_LICENSE"].astype(int)
df_model["SUSPICIOUS_CLAIM_TIME"] = df_model["SUSPICIOUS_CLAIM_TIME"].astype(int)
df_model["SUSPICIOUS_MILEAGE"] = df_model["SUSPICIOUS_MILEAGE"].astype(int)
df_model["EXCESSIVE_CLAIM_AMOUNT"] = df_model["EXCESSIVE_CLAIM_AMOUNT"].astype(int)

In [20]:
raw_df.groupby("FLAG_FOR_FRAUD_INV", as_index=False).mean()

,FLAG_FOR_FRAUD_INV,INCIDENT_CAUSE,DESCRIPTION,CLAIM_STATUS,ODOMETER_AT_LOSS,POLICE_REPORT,CLAIMS_AT_LOSS_DATE,LOSS_LOCATION_LAT,LOSS_LOCATION_LONG,CLAIM_AMOUNT,...,SUSPICIOUS_CLAIM_TIME,DAYS_FROM_L_EXPIRY,EXPIRED_LICENSE,DAYS_AT_ADDRESS,SUSPICIOUS_LIVING,LENGTH_OF_POLICY,MILES/YEAR,LOW_MILEAGE_AT_LOSS,SUSPICIOUS_MILEAGE,EXCESSIVE_CLAIM_AMOUNT
0,0,2.926186,NaN,2.977153,180783.846046,0.281195,1.492091,41.857895,-87.675014,2521.147627,...,0.933216,894.546573,0.042179,3487.956063,0.996485,1.070159,205170.356701,0.0,0.094903,0.086116
1,1,3.041872,NaN,1.103448,175504.046305,0.637931,1.758621,41.860133,-87.679617,26429.480296,...,0.945813,917.960591,0.036946,3252.413793,1.000000,0.960490,230687.578997,0.0,0.088670,0.933498


In [21]:
#split data into x and y variables
xVar = df_model[["EXPIRED_LICENSE","SUSPICIOUS_CLAIM_TIME","SUSPICIOUS_LIVING","SUSPICIOUS_MILEAGE","EXCESSIVE_CLAIM_AMOUNT"]]
yVar = df_model["FLAG_FOR_FRAUD_INV"]

In [22]:
xVar.head()

,EXPIRED_LICENSE,SUSPICIOUS_CLAIM_TIME,SUSPICIOUS_LIVING,SUSPICIOUS_MILEAGE,EXCESSIVE_CLAIM_AMOUNT
0,0,1,1,0,1
1,1,1,1,0,0
2,0,1,1,1,1
3,0,1,1,0,1
4,0,1,1,0,0


In [23]:
#split into a test/train set
X_train, X_test, y_train, y_test = train_test_split(xVar, yVar, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(780, 5) (780,)
(195, 5) (195,)


In [24]:
#train model
clf = RandomForestClassifier(n_jobs=2, random_state=0)

clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [25]:
#create confusion matrix to gut check model
preds = clf.predict(X_test)
pd.crosstab(y_test, preds, rownames=['Actual Result'], colnames=['Predicted Result'])


Predicted Result,0,1
Actual Result,,
0,104,5
1,5,81
